# Projeto Final | Analytics Engineering
----
**Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro**

# Etapa 1 (Landing e Bronze):

**Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

## Landing - Extração de dados

In [ ]:
## Importar modulo para extração dos dados
from modules.etl import Landing

url = [
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz'
]
data_path = './data/'
landing_step = Landing(landing_path=data_path)
landing_step.extract(url=url)
landing_step.transform()

## Preparação do Datalake

In [ ]:
from modules.database import criar_database, criar_schemas

database = 'datalake'
criar_database(database=database)
criar_schemas(database=database)

## Bronze - Ingestão e Análise dos Dados

In [ ]:
from pandas import read_csv
from modules.database import criar_tabela_df

# Testar a conexão
try:
    print("Iniciando a estapa bronze.")

    ## Listings
    data_path = './data/'
    df_listings = read_csv(data_path + 'listings.csv') # Carregando o csv listings em um Dataframe Python
    criar_tabela_df('datalake','bronze', 'listings', df_listings,'replace') # Criar e salvar o DataFrame na tabela "listings" dentro do esquema "bronze"
    del df_listings

    ## Reviews
    df_reviews = read_csv(data_path + 'reviews.csv') # Carregando o csv reviews em um Dataframe Python
    criar_tabela_df('datalake','bronze', 'reviews', df_reviews,'replace') # Criar e salvar o DataFrame na tabela "reviews" dentro do esquema "bronze"
    del df_reviews

    ## Calendar
    df_calendar = read_csv(data_path + 'calendar.csv') # Carregando o csv calendar em um Dataframe Python
    criar_tabela_df('datalake','bronze', 'calendar', df_calendar,'replace') # Criar e salvar o DataFrame na tabela "calendar" dentro do esquema "bronze"
    del df_calendar

    print("Etapas bronze executada com sucesso!!")
except Exception as e:
    print("Ocorreu um erro inesperado ao efetua a etapa bronze:", e)

# Etapa 2 (Silver):

**Data Clean - Camada Silver**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

### **Listings**

#### 0. Importar

In [1]:
import pandas as pd
from modules.database import engine_db
from modules.utils import remove_special_caracters, process_dataframe, remove_outliers, change_t_for_1 # Bibliotecas de limpeza

# Leitura das tabelas bronze
database = 'datalake'
df_silver_listings = pd.read_sql_table(table_name='listings',schema='bronze',con=engine_db(database=database))

c:\Users\rafael.qsantos\Documents\Engenharia de Dados\ada_enginnering_analytics\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 1. Relatório

In [2]:
from modules.utils import data_profiling
#data_profiling(df_silver_listings, 'listings', 'data_profiling/')

#### 2. Processamento

In [5]:
# Limpeza de caracteres
df_silver_listings["neighborhood_overview"] = df_silver_listings["neighborhood_overview"].apply(remove_special_caracters)
df_silver_listings["host_about"] = df_silver_listings["host_about"].apply(remove_special_caracters)

# Identificar e lidar com valores ausentes, duplicatas e outliers
cols_listings = ['review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']
df_process = process_dataframe(df_silver_listings, 'listings')
df_silver_listings = remove_outliers(df_process, cols_listings)

# Conversão do T e F para 1 e 0
boolean_columns = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable']
for col in boolean_columns: # Iterando coluna por coluna para efetuar a conversão
    df_silver_listings[col] = df_silver_listings[col].apply(change_t_for_1).astype("int")

# Transformação da coluna bathroom_text em 2 colunas
df_silver_listings['bathrooms_text'] = df_silver_listings['bathrooms_text'].fillna('0 bath') # Preencha os valores nulos na coluna 'bathrooms_text' com '0 bath'
df_silver_listings['bathroom_type'] = df_silver_listings['bathrooms_text'].str.replace(r'(\d+\.?\d*)', '', regex=True) # Defina o tipo de banheiro
df_silver_listings['bathroom_type'] = df_silver_listings['bathroom_type'].str.replace(r's?$', '', regex=True).apply(lambda x: x.strip()) # Remova 's' do final de 'baths e traz o strip()'
df_silver_listings['bathroom_quantity'] = df_silver_listings['bathrooms_text'].str.extract(r'(\d+\.?\d*)')[0].astype(float) # Extraia o número de banheiros usando expressões regulares
df_silver_listings['bathroom_quantity'] = pd.to_numeric(df_silver_listings['bathroom_quantity'], errors='coerce').fillna(0) # Converta a coluna 'bathroom_quantity' para tipo numérico

## Padronização dos nomes e tipos das colunas
# Definir o tipo das colunas que não serão de texto
list_int_listings = ['id', 'host_id', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']
list_datetime_listings = ['last_scraped']
list_float_listings = ['review_scores_location', 'review_scores_value', 'reviews_per_month']
df_silver_listings['last_scraped'] = pd.to_datetime(df_silver_listings['last_scraped'], format='%Y-%m-%d')

Analisando a tabela listings

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  listings : 
 ['bathrooms', 'bedrooms', 'description', 'calendar_updated', 'license', 'neighbourhood_group_cleansed']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela listings

Removendo colunas constantes ...
Colunas constantes removidas da tabela listings: 
 ['scrape_id', 'amenities']

Quantidade de linhas (outliers) eliminadas:  27265


### **Reviews**

#### 0. Importar

In [3]:
import pandas as pd
from modules.database import engine_db
from modules.utils import remove_special_caracters, process_dataframe, remove_outliers, change_t_for_1 # Bibliotecas de limpeza

# Leitura das tabelas bronze
database = 'datalake'
df_silver_reviews = pd.read_sql_table(table_name='reviews',schema='bronze',con=engine_db(database=database))

#### 1. Relatório

In [4]:
from modules.utils import data_profiling
#data_profiling(df_silver_reviews, 'reviews', 'data_profiling/')

#### 2. Processamento

In [12]:
# Identificar e lidar com valores ausentes, duplicatas e outliers
df_silver_reviews["comments"] = df_silver_reviews["comments"].apply(remove_special_caracters) # Limpeza de caracteres
df_process = process_dataframe(df_silver_reviews, 'reviews')
# df_silver_reviews = remove_outliers(df_process, cols_reviews)

# Padronização dos nomes e tipos das colunas
list_of_columns_int_reviews = ['listing_id', 'id', 'reviewer_id']
list_of_columns_datetime_reviews = ['date']
df_silver_reviews['date'] = pd.to_datetime(df_silver_reviews['date'], format='%Y-%m-%d')

Analisando a tabela reviews

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  reviews : 
 []

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela reviews

Removendo colunas constantes ...
Colunas constantes removidas da tabela reviews: 
 []


### **Calendar**

#### 0. Importar

In [1]:
import pandas as pd
from modules.database import engine_db
from modules.utils import remove_special_caracters, process_dataframe, remove_outliers, change_t_for_1 # Bibliotecas de limpeza

# Leitura das tabelas bronze
database = 'datalake'
df_silver_calendar = pd.read_sql_table(table_name='calendar',schema='bronze',con=engine_db(database=database))

#### 1. Relatório

In [6]:
from modules.utils import data_profiling
#data_profiling(df_silver_calendar, 'calendar', 'data_profiling/')

#### 2. Processamento

In [14]:
df_silver_calendar["price"] = df_silver_calendar["price"].apply(remove_special_caracters).astype("float") # Limpeza de caracteres

## Identificar e lidar com valores ausentes, duplicatas e outliers
cols_calendar = ['price'] # colunas para considerar a eliminação de outliers
df_process = process_dataframe(df_silver_calendar, 'calendar')
df_silver_calendar = remove_outliers(df_process, cols_calendar)

## Conversão do T e F para 1 e 0
boolean_columns = ['available']
for col in boolean_columns:
    df_silver_calendar[col] = df_silver_calendar[col].apply(change_t_for_1).astype("int")

## Padronização dos nomes e tipos das colunas
# Definir o tipo das colunas que não serão de texto
list_int_calendar = ['listing_id', 'minimum_nights', 'maximum_nights']
list_datetime_calendar = ['date']
list_float_calendar = ['price']
# Alterar o tipo das colunas 'date','minimum_nights', 'maximum_nights'
df_silver_calendar['date'] = pd.to_datetime(df_silver_calendar['date'], format='%Y-%m-%d')
for column in ['minimum_nights', 'maximum_nights']:
    df_silver_calendar[column] = df_silver_calendar[column].astype("int") #transforma a coluna para o tipo inteiro

Analisando a tabela calendar

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  calendar : 
 ['adjusted_price']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela calendar

Removendo colunas constantes ...
Colunas constantes removidas da tabela calendar: 
 []

Quantidade de linhas (outliers) eliminadas:  551925


C:\Users\rafael.qsantos\AppData\Local\Temp\ipykernel_72652\4241121525.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar[col] = df_silver_calendar[col].apply(change_t_for_1).astype("int")
C:\Users\rafael.qsantos\AppData\Local\Temp\ipykernel_72652\4241121525.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver_calendar['date'] = pd.to_datetime(df_silver_calendar['date'], format='%Y-%m-%d')


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

# Etapa 3:

**Data Quality - Camada Silver**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

## **Listings**

### Processamento

In [ ]:
from modules.utils import check_missing
 
check_missing(df_silver_listings).sort_values(ascending=False) # Todas as colunas com dados faltantes

### Preenchimento dos campos nulos com valores fixos
- df_cln['notRepairedDamage'] = df_cln['notRepairedDamage'].fillna("no_info")

### Preenchimento dos campos nulos com o campo que mais se repete
- high_freq = df_cln['fuelType'].value_counts().idxmax()
- df_cln['fuelType'] = df_cln['fuelType'].fillna(high_freq)

### Preenchimento dos campos nulos com valores fixos de outra coluna
- df_cln['model'] = df_cln['model'].fillna(df_cln['vehicleType'])

### Verificação da precisão dos dados
ALGUMAS POSSÍVEIS VERIFICAÇÕES:

- Os meses precisam estar no intervalo: 1 <= meses <= 12
- Os anos precisam estar no intervalo: 1900 <= ano <= 2016
```python
df_cln.loc[(df_cln['yearOfRegistration']<1900) | (df_cln['yearOfRegistration']>2016), 'yearOfRegistration'] = 1900
```

- Os preços precisam ser maiores que 0

# Etapa 4:

**Testes de Qualidade - Camada Silver**

   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

# Etapa 5:

**Transformação de Dados com dbt - Camada Silver**

   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

# Etapa 6:

**Armazenamento de Dados em PostgreSQL - Camada Silver**

   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

# Etapa 7:

**Validação de Expectativas com Great Expectations - Camada Silver**

   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

# Etapa 8:

**Transformação de Dados com dbt - Camada Gold**

   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.